In [ ]:
import json
import os

# JSON ファイルのパス（必要に応じてパスを調整してください）
json_path = "dataset/test_split.json"

# 出力ディレクトリの作成（存在しない場合）
os.makedirs("PLU", exist_ok=True)

# JSON ファイルを読み込み
with open(json_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# "[Determine Substrate]" の例だけを抽出
filtered = [entry for entry in data if entry["instruction"] == "[Determine Substrate]"]

# PLU/input.txt 用：instruction と input (ここでは "Seq=<...>" の部分) をそのまま出力
with open("PLU/input.txt", "w", encoding="utf-8") as f_in:
    for entry in filtered:
        line = f'{entry["instruction"]} {entry["input"]}'
        f_in.write(line + "\n")

# PLU/true.txt 用：instruction, input, output を1行にまとめて出力
with open("PLU/true.txt", "w", encoding="utf-8") as f_true:
    for entry in filtered:
        line = f'{entry["instruction"]} {entry["input"]} {entry["output"]}'
        f_true.write(line + "\n")

print("PLU/input.txt と PLU/true.txt の出力が完了しました。")

In [ ]:
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer, GenerationConfig
from tqdm import tqdm
import os

# 生成設定（必要に応じて調整してください）
generation_config = GenerationConfig(
    temperature=0.2,
    top_k=40,
    top_p=0.9,
    do_sample=True,
    num_beams=1,
    repetition_penalty=1.2,
    max_new_tokens=900
)

In [ ]:
# GPU チェック
load_type = torch.bfloat16
if torch.cuda.is_available():
    device = torch.device(0)
else:
    raise ValueError("No GPU available.")

# モデルパスの指定
model_path = 'stored_output/checkpoint-2epoch'
model = LlamaForCausalLM.from_pretrained(
    model_path,
    torch_dtype=load_type,
    low_cpu_mem_usage=True,
    device_map='auto',
    quantization_config=None
)
model.eval()
tokenizer = LlamaTokenizer.from_pretrained(model_path)

In [ ]:
outputs = []
# 入力ファイルと出力ファイルのパス
input_file = "PLU/input.txt"
output_file = "PLU/pred.txt"
# 出力先ディレクトリが存在しない場合は作成
os.makedirs(os.path.dirname(output_file), exist_ok=True)
# 入力ファイルの読み込み
with open(input_file, 'r', encoding='utf-8') as f:
    examples = f.read().splitlines()

# バッチサイズの設定
batch_size = 8
tokenizer.padding_side = 'left'

print("Start generating predictions...")
# バッチ処理の実装
for i in tqdm(range(0, len(examples), batch_size), desc="Processing batches"):
    # 現在のバッチを取得
    batch_examples = examples[i:i + batch_size]
    
    # バッチ内の例をトークナイズ
    batch_inputs = tokenizer(batch_examples, padding=True, truncation=True, return_tensors="pt")
    
    # モデルによる生成
    batch_outputs = model.generate(
        input_ids=batch_inputs["input_ids"].to(device),
        attention_mask=batch_inputs['attention_mask'].to(device),
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        generation_config=generation_config
    )
    
    # 生成結果のデコード
    for j in range(len(batch_examples)):
        output = tokenizer.decode(batch_outputs[j], skip_special_tokens=True)
        output = output.replace("</s>", "")
        outputs.append(output)

In [ ]:
# 予測結果の書き出し
with open(output_file, 'w', encoding='utf-8') as f:
    f.write("\n".join(outputs))

print("All the outputs have been saved in", output_file)

In [1]:
import re

def extract_set(line):
    """
    文字列から "Substrate=<...>" 部分を抽出し、カンマ区切りで分割した要素の集合を返す。
    例: "[Determine Substrate] Seq=<...> Substrate=<chloride, hydron>" -> {"chloride", "hydron"}
    """
    match = re.search(r"Substrate=<(.+?)>", line)
    if match:
        content = match.group(1)
        # カンマで分割し、前後の空白を除去して小文字に（必要に応じて正規化）
        elements = {elem.strip().lower() for elem in content.split(",") if elem.strip()}
        return elements
    return set()

# ファイルの読み込み
with open("PLU/true.txt", "r", encoding="utf-8") as f_true, open("PLU/pred.txt", "r", encoding="utf-8") as f_pred:
    true_lines = f_true.read().splitlines()
    pred_lines = f_pred.read().splitlines()

if len(true_lines) != len(pred_lines):
    print(f"行数が一致しません: true: {len(true_lines)}, pred: {len(pred_lines)}")
    # 行数が異なる場合、少ない方に合わせて計算する
    N = min(len(true_lines), len(pred_lines))
    true_lines = true_lines[:N]
    pred_lines = pred_lines[:N]
else:
    print(f"行数が一致しました: {len(true_lines)}")
    N = len(true_lines)

# 指標用の累積変数
total_intersection = 0 # 予測と真の集合の積集合の要素数 Σ|Fi ∩ F'i|
total_pred_count = 0 # 予測の集合の要素数 Σ|F'i|
total_true_count = 0 # 真の集合の要素数 Σ|Fi|
total_union = 0 # 予測と真の集合の和集合の要素数 Σ|Fi ∪ F'i|
exact_match_count = 0  # 予測と真の集合が正確一致する行数 Σ δ(Fi, F'i)
matched_indices = [] # 予測と真の集合が正確一致する行のインデックス
mismatched_indices = [] # 予測と真の集合が正確一致しない行のインデックス

# 各行ごとに評価
for i, (t_line, p_line) in enumerate(zip(true_lines, pred_lines)):
    true_set = extract_set(t_line)
    pred_set = extract_set(p_line)
    
    intersection = true_set.intersection(pred_set)
    union = true_set.union(pred_set)
    
    total_intersection += len(intersection)
    total_pred_count += len(pred_set)
    total_true_count += len(true_set)
    total_union += len(union)
    
    # 一致判定とインデックスの格納
    if true_set == pred_set:
        exact_match_count += 1
        matched_indices.append(i)
    else:
        mismatched_indices.append(i)

# 各指標の計算
precision = total_intersection / total_pred_count if total_pred_count > 0 else 0 # 適合率 = Σ|Fi ∩ F'i| / Σ|F'i|
recall = total_intersection / total_true_count if total_true_count > 0 else 0 # 再現率 = Σ|Fi ∩ F'i| / Σ|Fi|
f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0 # F1スコア = 2 * (適合率 * 再現率) / (適合率 + 再現率)
jaccard = total_intersection / total_union if total_union > 0 else 0 # Jaccard係数 = Σ|Fi ∩ F'i| / Σ|Fi ∪ F'i|
exact_match = exact_match_count / N # 正確一致率 = Σ δ(Fi, F'i) / N

print("Evaluation Metrics:")
print(f"Precision: {precision:.4f} ({total_intersection}/{total_pred_count})")
print(f"Recall: {recall:.4f} ({total_intersection}/{total_true_count})")
print(f"F1-Score: {f1:.4f}")
print(f"Jaccard-Similarity: {jaccard:.4f} ({total_intersection}/{total_union})")
print(f"Exact-Match: {exact_match:.4f} ({exact_match_count}/{N})")

行数が一致しました: 10684
Evaluation Metrics:
Precision: 0.9866 (39186/39719)
Recall: 0.9519 (39186/41164)
F1-Score: 0.9690
Jaccard-Similarity: 0.9398 (39186/41697)
Exact-Match: 0.9583 (10239/10684)


In [ ]:
# 不一致行のサンプルを表示（最初の10件）
if len(mismatched_indices) > 0:
    print("\n=== 不一致行のサンプル (最初の10件) ===")
    for i, idx in enumerate(mismatched_indices[:10]):
        print(f"インデックス {idx}:")
        print(f"  正解: {true_lines[idx]}")
        print(f"  予測: {pred_lines[idx]}")
        print()